In [0]:
%sh sudo apt-get update && apt-get install -y gdal-bin libeccodes-dev libnetcdf-dev libhdf5-dev

In [0]:
%pip install numpy==1.24.4 eccodes==1.7.1 xarray==2024.9.0 cf_xarray==0.10.0 cfgrib==0.9.14.1 netcdf4==1.7.2
%pip install --no-cache-dir --force-reinstall cfgrib==0.9.14.1 numpy==1.24.4

In [0]:
import xarray as xr

In [0]:
variables = [file_info.name[:-1] for file_info in dbutils.fs.ls("/Volumes/cdp_prd_sandbox_catalog_01/weather_silver/silver/era5/ERA5SH/")]
print(variables)

In [0]:
path_template = "/Volumes/cdp_prd_sandbox_catalog_01/weather_silver/silver/era5/ERA5SH/<variable>/2025/04/16/"
filepath_template = f"{path_template}ERA5SH_0p25_<variable>_2025041610.nc"

In [0]:
dims = []
datasets = {}
for var in variables:
    file_path = filepath_template.replace("<variable>", var)
    ds = xr.open_dataset(file_path)
    datasets[var] = ds
    dims.append(ds.dims)

set([str(dim) for dim in dims])

In [0]:
datasets

In [0]:
import matplotlib.pyplot as plt

def draw_map(data, title):
    lats = data['latitude'].values
    lons = data['longitude'].values
    values = data.values   # The actual data to plot

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(lons, lats, values, shading='auto', cmap='viridis')  # or use plt.imshow for regular grids
    plt.colorbar(label='Your Variable Units')

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(title)

    plt.show()


In [0]:
for var, ds in datasets.items():
    draw_map(ds[var], f"Map of {var}")

In [0]:
for var, ds in datasets.items():
    print(f"\n\n{var}")
    print(ds[var].values)

In [0]:
for var, ds in datasets.items():
    print(f"{var} ============\n\n")
    print(ds)


In [0]:
datasets["itssrd"]

In [0]:
%pip install dask

In [0]:
import xarray as xr
import glob

path = "/Volumes/cdp_prd_sandbox_catalog_01/weather_silver/silver/era5/ERA5SH/itssrd/2025/04/16/"
file_pattern = "ERA5SH_0p25_itssrd_*.nc"

files = sorted(glob.glob(path + file_pattern))

datasets = [xr.open_dataset(f) for f in files]

ds = xr.concat(datasets, dim="time") 
print(ds)

In [0]:
print("\nValid time values:")
print(ds['valid_time'].values)

In [0]:
#read grib
file_path = "/Volumes/cdp_prd_sandbox_catalog_01/weather_bronze/bronze/era5/single_hourly/2025/04/16/20250416_surface_solar_radiation_downwards.grib"

ds_grib = xr.open_dataset(file_path, engine="cfgrib", backend_kwargs={"indexpath": ""})

print(ds_grib)

In [0]:
# Print time values
print("Time values:")
print(ds_grib['time'].values)

# Print step values
print("\nStep values:")
print(ds_grib['step'].values)

step_hours = ds_grib['step'].values.astype('timedelta64[h]').astype(int)
print("\nStep values in hours:")
print(step_hours)

print("\nValid time values:")
print(ds_grib['valid_time'].values)